<a href="https://colab.research.google.com/github/jjh599/jackjohnhardy.github.io/blob/main/OxfordFlowers102CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras import layers
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
#device_name = tf.test.gpu_device_name()
#if device_name != '/device:GPU:0':
#  raise SystemError('GPU device not found')
#print('Found GPU at: {}'.format(device_name))

dataset, dataset_info = tfds.load('oxford_flowers102', with_info=True, as_supervised=True)
dataset_info
test_set, training_set, validation_set = dataset['test'], dataset['train'], dataset['validation']

In [2]:
import tensorflow as tf
tf.config.experimental.list_physical_devices('GPU')
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
  except RuntimeError as e:
    print(e)

print(gpus)

Physical devices cannot be modified after being initialized
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


Importing TensorFlow 

In [3]:
num_classes = dataset_info.features['label'].num_classes
num_training_examples = 1020
num_validation_examples = 1020

In [4]:
IMAGE_RES = 224

def format_image(image, label):
    image = tf.image.resize(image, (IMAGE_RES, IMAGE_RES))/255.0
    #label = tf.one_hot(label, depth=num_classes)
    return image, label
BATCH_SIZE = 32
train_batches = training_set.shuffle(num_training_examples//4).map(format_image).batch(BATCH_SIZE).prefetch(1)
validation_batches = validation_set.shuffle(num_validation_examples//4).map(format_image).batch(BATCH_SIZE).prefetch(1)
test_batches = test_set.map(format_image).batch(BATCH_SIZE).prefetch(1)


In [5]:
def make_model(input_shape, num_classes):
  model = tf.keras.Sequential()
  model.add(layers.RandomFlip("horizontal_and_vertical", input_shape=input_shape))
  model.add(layers.RandomRotation(20))
  model.add(layers.RandomZoom(0.2))
  #model.add(layers.RandomContrast(factor=(0.0, 0.1)))
  model.add(layers.Conv2D(64, 3, padding='same', activation='relu')) #64 filters, kernel size 3x3, stride = 1
  model.add(layers.BatchNormalization())
  model.add(layers.Conv2D(64, 3, padding='same', activation='relu')) 
  model.add(layers.BatchNormalization())
  model.add(layers.MaxPooling2D())                                   #2x2 pooling, stride of 2
  model.add(layers.Conv2D(128, 3, padding='same', activation='relu'))
  model.add(layers.BatchNormalization())
  model.add(layers.Conv2D(128, 3, padding='same', activation='relu'))
  model.add(layers.BatchNormalization())
  model.add(layers.MaxPooling2D())
  model.add(layers.Conv2D(256, 3, padding='same', activation='relu'))
  model.add(layers.BatchNormalization())
  model.add(layers.Conv2D(256, 3, padding='same', activation='relu'))
  model.add(layers.BatchNormalization())
  model.add(layers.Conv2D(256, 3, padding='same', activation='relu'))
  model.add(layers.BatchNormalization())
  model.add(layers.MaxPooling2D())
  model.add(layers.Conv2D(512, 3, padding='same', activation='relu'))
  model.add(layers.BatchNormalization())
  model.add(layers.Conv2D(512, 3, padding='same', activation='relu'))
  model.add(layers.BatchNormalization())
  model.add(layers.Conv2D(512, 3, padding='same', activation='relu'))
  model.add(layers.BatchNormalization())
  model.add(layers.MaxPooling2D())
  model.add(layers.Conv2D(512, 3, padding='same', activation='relu'))
  model.add(layers.BatchNormalization())
  model.add(layers.Conv2D(512, 3, padding='same', activation='relu'))
  model.add(layers.BatchNormalization())
  model.add(layers.Conv2D(512, 3, padding='same', activation='relu'))
  model.add(layers.BatchNormalization())
  model.add(layers.MaxPooling2D())
  model.add(layers.Flatten())
  model.add(layers.Dense(4096, activation='relu'))
  model.add(layers.Dropout(0.5))
  model.add(layers.Dense(4096, activation='relu'))
  model.add(layers.Dropout(0.5))
  model.add(layers.Dense(1024, activation='relu'))
  model.add(layers.Dropout(0.5))
  model.add(layers.Dense(num_classes))
  return model


In [6]:
tf.keras.backend.clear_session()

In [7]:
model = make_model(input_shape=(IMAGE_RES, IMAGE_RES) + (3,), num_classes=102)
#keras.utils.plot_model(model, show_shapes=True)

epochs = 50

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.000001)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

model.fit(
    train_batches,
    epochs=epochs,
    validation_data=validation_batches,
    batch_size=BATCH_SIZE,
    validation_batch_size=BATCH_SIZE,
    verbose=1,
    callbacks=[reduce_lr]
)

Epoch 1/50
32/32 [==============================] - 58s 1s/step - loss: 8.6423 - accuracy: 0.0108 - val_loss: 4.6294 - val_accuracy: 0.0098 - lr: 1.0000e-04
Epoch 2/50
32/32 [==============================] - 21s 641ms/step - loss: 5.4733 - accuracy: 0.0127 - val_loss: 4.6325 - val_accuracy: 0.0098 - lr: 1.0000e-04
Epoch 3/50
32/32 [==============================] - 21s 661ms/step - loss: 5.0890 - accuracy: 0.0176 - val_loss: 4.6374 - val_accuracy: 0.0098 - lr: 1.0000e-04
Epoch 4/50
32/32 [==============================] - 21s 636ms/step - loss: 4.9040 - accuracy: 0.0245 - val_loss: 4.6482 - val_accuracy: 0.0088 - lr: 1.0000e-04
Epoch 5/50
32/32 [==============================] - 21s 653ms/step - loss: 4.8157 - accuracy: 0.0284 - val_loss: 4.6774 - val_accuracy: 0.0098 - lr: 1.0000e-04
Epoch 6/50
32/32 [==============================] - 21s 652ms/step - loss: 4.6591 - accuracy: 0.0275 - val_loss: 4.6630 - val_accuracy: 0.0098 - lr: 1.0000e-04
Epoch 7/50
32/32 [=========================

In [15]:
model.save('OxfordFlowers102-2.keras')

In [8]:
test_loss, test_acc = model.evaluate(test_batches, verbose=1, batch_size=BATCH_SIZE)
print("Test accuracy:", test_acc)

193/193 [==============================] - 27s 140ms/step - loss: 3.2010 - accuracy: 0.2420
Test accuracy: 0.2419905662536621
